<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Normalizacja" data-toc-modified-id="Normalizacja-1">Normalizacja</a></span><ul class="toc-item"><li><span><a href="#Wartości-odstające" data-toc-modified-id="Wartości-odstające-1.1">Wartości odstające</a></span></li><li><span><a href="#Klasyfikacja" data-toc-modified-id="Klasyfikacja-1.2">Klasyfikacja</a></span></li>

## Normalizacja

### Wartości odstające

**Obserwacja odstająca**, element odstający (ang. **outlier**) – obserwacja relatywnie odległa od pozostałych elementów próby.

https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Elements_of_a_boxplot_en.svg/2560px-Elements_of_a_boxplot_en.svg.png

**IQR** (od ang. interquartile range) - rozstęp ćwiartkowy, rozstęp międzykwartylowy - różnica między trzecim a pierwszym kwartylem.

In [ ]:
import numpy as np
from scipy.stats import iqr

### Klasyfikacja

Wykorzystamy zestaw danych dotyczącego win. Niestety, klasy są nieopisane, więc nie możemy nadać sensownej interpretacji tym klasom, ale wnioskować można, czy klasy są możliwe do opisania na podstawie danych.

In [ ]:
from sklearn.datasets import load_wine

import pandas as pd
import numpy as np

from sklearn import preprocessing
from matplotlib import pyplot as plt

In [ ]:
data = load_wine()
print(data.DESCR)

In [ ]:
# Przygotowanie pandasowej ramki danych
df = pd.DataFrame(data.data, columns = data.feature_names)
df.head()

In [ ]:
cols = ['alcohol', 'malic_acid']
## Normalizacja do rozkładu normalnego (wiekszość algorytmów wymaga takiego dostosowania)
std_scale = preprocessing.StandardScaler().fit(df[cols])
df_std = std_scale.transform(df[cols])

## Normalizacja do zakresu (podatne na wartości odstające, nieco prostsze obliczeniowo)
minmax_scale = preprocessing.MinMaxScaler().fit(df[cols])
df_minmax = minmax_scale.transform(df[cols])

Normalizacja jest niezbędnym krokiem. Miary mają sens, gdy zakładamy równą ważność każdej zmiennej, a jeżeli zmienne mają różną skalę, ich wpływ na wartość funkcji dystansu jest zróznicowany - co powoduje, że konkretna zmienna będzie bardziej istotna w klasyfikacji. Aby uniknąć tego efektu (czasem może on być pożądany), należy doprowadzić do sytuacji, gdzie wszystkie zmienne mają taką samą skalę. Proces ten nazywamy normalizacją.

Najczęstsze dwa sposoby normalizacji danych to liniowe skalowanie każdej zmiennej do określonego zakresu (skalowanie min-max, zaimplementowane przez klasę `MinMaxScaler`), oraz przeskalowanie danych w taki sposób, aby miały średnią i wariancję odpowiednio (0, 1). Ten drugi sposób nazywa się standaryzacją danych.

Zobaczmy, jak będą wyglądały statystyki dwóch wybranych zmiennych po zastosowaniu normalizacji.

In [ ]:
print('original \n', pd.DataFrame(df[cols]).describe(), '\n')
print('min max \n', pd.DataFrame(df_minmax).describe(), '\n')
print('z score \n', pd.DataFrame(df_std).describe(), '\n')

Aby poprawić naszą intuicję, zwizualizujmy jak układają się znormalizowane zestawy danych na układzie współrzędnych. Zauważ, jakie są centra każdej z chmur punktów oraz jakie są granice danych znormalizowanych przez min-max.

In [ ]:
# Jak poszczególne metryki wpływają na zachowanie naszego zbioru danych?

plt.figure(figsize=(12,8))

plt.scatter(df['alcohol'], df['malic_acid'],
        color='green', label='input scale', alpha=0.5)

plt.scatter(df_std[:,0], df_std[:,1], color='red',
        label='Standardized', alpha=0.3)

plt.scatter(df_minmax[:,0], df_minmax[:,1],
        color='blue', label='min-max scaled', alpha=0.3)

plt.title('alcohol and malic_acid content of the wine dataset')
plt.xlabel('alcohol')
plt.ylabel('malic_acid')
plt.legend(loc='upper left')
plt.grid()

plt.tight_layout()
plt.show()

Istotną wadą min-max jest podatność na wartości odstające. Zwróć uwagę na to, jak dodanie nowej obserwacji bardzo mocno ogranicza zakres wartości min-max, natomiast prawie nie wpływa na ustandaryzowaną chmurę punktów.

In [ ]:
# Dodajmy punkt odstający
new_df = df[cols].append([{"alcohol": 20, "malic_acid": 8}])

new_df_std = preprocessing.StandardScaler().fit_transform(new_df)
new_df_minmax = preprocessing.MinMaxScaler().fit_transform(new_df)

In [ ]:
plt.figure(figsize=(12,8))

plt.scatter(new_df['alcohol'], new_df['malic_acid'],
        color='green', label='input scale', alpha=0.5)

plt.scatter(new_df_std[:,0], new_df_std[:,1], color='red',
        label='Standardized', alpha=0.3)

plt.scatter(new_df_minmax[:,0], new_df_minmax[:,1],
        color='blue', label='min-max scaled', alpha=0.3)

plt.title('alcohol and malic_acid content of the wine dataset')
plt.xlabel('alcohol')
plt.ylabel('malic_acid')
plt.legend(loc='upper left')

plt.grid()

plt.tight_layout()
plt.show()

Procedurą, którą należałoby zastosować jest odcięcie wartości odstających - da to pewność, że większość danych znajdzie się w oczekiwanym zakresie danych. Najprostszym sposobem jest odcięcie wartości wzdłuż każdej osi, które nie mieszczą się w określonym zakresie.

Na naszych zajęciach poznamy dwie metody - poniżej opisane jest odcinanie wartości leżących powyżej kilku odchyleń standardowych w naszym zestawie danych oraz metoda oparta o dystans między kwartylami (dla przypomnienia, pierwszy i trzeci kwartyl to wartości, które są większe lub równe 25% i 75% danych).

Istnieją także bardziej zaawansowane metody odcinania wartości odstających, opisane tu: https://scikit-learn.org/stable/modules/outlier_detection.html

In [ ]:
### Usunięcie wartości odstających - trzy odchylenia standardowe

def rm_out(df):
    df_out = df.copy()
    means, stdevs = {}, {}
    for column in df_out.columns:
        means[column] = df_out[column].mean()
        stdevs[column] = df_out[column].std()
    for column in df_out.columns:
        lower = means[column] - 3 * stdevs[column]
        upper = means[column] + 3 * stdevs[column]
        index = (lower < df_out[column]) & (df_out[column] < upper)
        df_out = df_out[index]
    df_out.index = range(len(df_out))
    return df_out

Usunięcie wartości odstających pozwala nam spodziewać się, że obydwa sposoby normalizacji przeskalują nasze dane we w miarę jednolity sposób.

In [ ]:
clipped_df = rm_out(new_df)
clipped_df_std = preprocessing.StandardScaler().fit_transform(clipped_df)
clipped_df_minmax = preprocessing.MinMaxScaler().fit_transform(clipped_df)

plt.figure(figsize=(12,8))
plt.scatter(clipped_df['alcohol'], clipped_df['malic_acid'],
        color='green', label='input scale', alpha=0.5)

plt.scatter(clipped_df_std[:,0], clipped_df_std[:,1], color='red',
        label='Standardized', alpha=0.3)

plt.scatter(clipped_df_minmax[:,0], clipped_df_minmax[:,1],
        color='blue', label='min-max scaled', alpha=0.3)

plt.title('alcohol and malic_acid content of the wine dataset')
plt.xlabel('alcohol')
plt.ylabel('malic_acid')
plt.legend(loc='upper left')
plt.grid()

plt.tight_layout()
plt.show()